In [4]:
import pandas as pd
import numpy as np
import os
import pickle

In [3]:
def load_data(filepath):
    # load list of json files
    with open(filepath, 'rb') as file:
        metadata_lst = pickle.load(file)

    return metadata_lst

In [5]:
csv = load_data('./saved_df/email_df')

In [6]:
csv

,file,message
427616,shackleton-s/sent/1912.,Message-ID: <21013688.1075844564560.JavaMail.e...
108773,farmer-d/logistics/1066.,Message-ID: <22688499.1075854130303.JavaMail.e...
355471,parks-j/deleted_items/202.,Message-ID: <27817771.1075841359502.JavaMail.e...
457837,stokley-c/chris_stokley/iso/client_rep/41.,Message-ID: <10695160.1075858510449.JavaMail.e...
124910,germany-c/all_documents/1174.,Message-ID: <27819143.1075853689038.JavaMail.e...
403283,scott-s/_sent_mail/244.,Message-ID: <10142547.1075846737160.JavaMail.e...
293966,love-p/discussion_threads/113.,Message-ID: <18212904.1075858229814.JavaMail.e...
478830,taylor-m/australia_trading/8.,Message-ID: <14840674.1075860237113.JavaMail.e...
295428,love-p/sent_items/765.,Message-ID: <22170097.1075862178026.JavaMail.e...
137822,giron-d/deleted_items/170.,Message-ID: <23520008.1075852220995.JavaMail.e...


In [101]:
filepath_metadata = './saved_metadata/metadata_lst'
filepath_usrdata = './saved_metadata/usr_info.csv'

metadata_lst = load_data(filepath_metadata)

In [102]:
# convert list of string jsons to dataframe
import json

metadata_cols = ['MessageID', 'Date', 'From', 'To', 'Cc', 'Bcc', 'Subject', 'MimeVersion',
                 'ContentType', 'ContentTransferEncoding', 'Summarized Content', 'Attachments' ]

metadata_df = pd.DataFrame(columns=metadata_cols)

for element in metadata_lst:
    metadata_json = json.loads(element)
    new_row = pd.DataFrame([metadata_json])
    metadata_df = pd.concat([metadata_df, new_row], ignore_index=True)


In [103]:
# Standardize date/time to EST
import datetime

datetime_lst = []

for date in metadata_df['Date']:
    # modify date to "%Y-%m-%d %H:%M:%S" format
    input_datetime = str(date)
    input_datetime = input_datetime[:-12]
    dt_object = datetime.datetime.strptime(input_datetime, "%a, %d %b %Y %H:%M:%S")
    output_datetime = dt_object.strftime("%Y-%m-%d %H:%M:%S")
    output_datetime = datetime.datetime.strptime(output_datetime, "%Y-%m-%d %H:%M:%S")

    # convert from pst to est
    time_diff = datetime.timedelta(hours=3)
    est_datetime = output_datetime - time_diff
    est_datetime = est_datetime.strftime("%Y-%m-%d %H:%M:%S")
    
    # append to date_lst
    datetime_lst.append(est_datetime)

# replace date with modified datetime format
metadata_df['Date'] = datetime_lst

In [104]:
# replace null or [] values with None
import re
metadata_df.replace([''], 'None', inplace=True)
metadata_df = metadata_df.applymap(lambda x: re.sub(r'^\[\s*\]$', 'None', str(x)))

In [119]:
# Load Logics
import sqlite3

# Create sqlite3 tables from dataframe
database_file_path = './email.db'
with sqlite3.connect(database_file_path) as conn:
    cur = conn.cursor()
    metadata_df.to_sql('Email', conn, if_exists='replace', index=False)
    conn.commit()
    print('Dataframe converted to table!')

# Add indexes after data is added


Dataframe converted to table!
Insert command successful!


In [120]:
# Insert the metadata_df
with sqlite3.connect(database_file_path) as conn:
    cur = conn.cursor()
    cur.execute('''
        INSERT INTO Email (MessageID, Date, "From", "To", Cc, Bcc, Subject, MimeVersion, ContentType, ContentTransferEncoding, "Summarized Content", Attachments)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    ''', (metadata_df.iloc[2]))
    
    conn.commit()
    print('Insert command successful!')


Insert command successful!


In [106]:
metadata_df

,MessageID,Date,From,To,Cc,Bcc,Subject,MimeVersion,ContentType,ContentTransferEncoding,Summarized Content,Attachments
0,<21013688.1075844564560.JavaMail.evans@thyme>,2000-08-28 22:26:00,sara.shackleton@enron.com,william.bradford@enron.com,None,None,Re: Credit Derivatives,1.0,text/plain; charset=us-ascii,7bit,Sara asks for clarification on EnronCredit.com...,None
1,<22688499.1075854130303.JavaMail.evans@thyme>,2000-04-24 02:43:00,pat.clynes@enron.com,aimee.lannou@enron.com,daren.farmer@enron.com,daren.farmer@enron.com,Meter #1591 Lamay Gaslift,1.0,text/plain; charset=us-ascii,7bit,Please check meter #1591 Lamay gas lift. It d...,None
2,<27817771.1075841359502.JavaMail.evans@thyme>,2002-05-02 01:54:27,knipe3@msn.com,"fenner.chet@enron.com, joe.parks@enron.com, co...",None,None,Re: man night again?,1.0,text/plain; charset=us-ascii,7bit,GCCA Crawfish and rip-off raffle & over-priced...,None
3,<10695160.1075858510449.JavaMail.evans@thyme>,2001-08-08 11:35:08,kalmeida@caiso.com,chris.stokley@enron.com,None,None,"Enron 480, 1480 charges",1.0,text/plain; charset=us-ascii,7bit,"Chris, per your request here are the 480 and 1...",['Keoni.zip']
4,<27819143.1075853689038.JavaMail.evans@thyme>,2000-06-21 01:58:00,chris.germany@enron.com,thomas.engel@enron.com,None,None,Transport Deal,1.0,text/plain; charset=us-ascii,7bit,Chris Germany is trying to change the Receipt ...,None
5,<10142547.1075846737160.JavaMail.evans@thyme>,2000-05-10 08:22:00,susan.scott@enron.com,david.foti@enron.com,rich.jolly@enron.com,rich.jolly@enron.com,Re: WT-1 Electric Service Agreement,1.0,text/plain; charset=us-ascii,7bit,Susan Scott suggests replacing Section 2 and 3...,None
6,<18212904.1075858229814.JavaMail.evans@thyme>,2000-06-07 08:30:00,phillip.love@enron.com,delma.salazar@enron.com,None,None,0003-0001 OAs as of 4/2000 GL,1.0,text/plain; charset=us-ascii,7bit,Forwarded email regarding 0003-0001 OAs as of ...,None
7,<14840674.1075860237113.JavaMail.evans@thyme>,2000-04-12 23:42:00,cynthia.harkness@enron.com,mark.taylor@enron.com,None,None,Leona Tan,1.0,text/plain; charset=us-ascii,7bit,Attached is Leona Tan's resume. Please let me ...,Leona Tan's resume
8,<22170097.1075862178026.JavaMail.evans@thyme>,2001-11-09 10:36:17,m..love@enron.com,stevebonilla@yahoo.com,None,None,RE: Hello,1.0,text/plain; charset=us-ascii,7bit,got your message last night. What is up? Bet...,None
9,<23520008.1075852220995.JavaMail.evans@thyme>,2001-10-15 12:11:42,orderdetails@buy.com,dgiron@enron.com,None,None,Your buy.com order is on its way,1.0,text/plain; charset=us-ascii,7bit,The email is a notification that items from or...,None


In [107]:
# SQL logics

# Insert

# Update

# Delete

# Single Select 

# Select Many